In [1]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store csv created in part one into a DataFrame
vacation_df = pd.read_csv('../WeatherPy/Output/weatherdata.csv') 
vacation_df

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,Komsomolets,53.7453,62.0708,16.63,94,100,3.76
1,Ostrovnoy,68.0531,39.5131,18.79,83,100,10.51
2,Barrow,71.2906,-156.7887,-11.20,92,1,3.31
3,Tasiilaq,65.6145,-37.6368,6.80,56,20,8.05
4,Punta Arenas,-53.1500,-70.9167,53.60,58,75,19.57
...,...,...,...,...,...,...,...
557,Vitim,59.4511,112.5578,17.38,97,100,1.28
558,Vernon,50.2581,-119.2691,46.00,70,48,4.61
559,Tshikapa,-6.4167,20.8000,69.30,95,24,1.81
560,Belyy Yar,53.6039,91.3903,21.20,86,0,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps: Source: https://nu.bootcampcontent.com/NU-Coding-Bootcamp/nu-chi-data-pt-02-2021-u-c/blob/master/01-Lesson-Plans/06-Python-APIs/3/Activities/06-Evr_Jupyter_Gmaps/Solved/gmaps.ipynb
gmaps.configure(api_key=gkey)



In [4]:
# Heatmap of humidity
locations = vacation_df[["Latitude", "Longitude"]].astype(float)
weight = vacation_df["Humidity"].astype(float)

fig = gmaps.figure()

#Adds heat to layer

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#Adds heat layer to map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
hotel_df = vacation_df.loc[(vacation_df['Max Temperature (F)'] <= 70) & (vacation_df['Max Temperature (F)'] > 65) & (vacation_df['Cloudiness'] < 5) & (vacation_df['Wind Speed (mph)'] < 10)]
hotel_df

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
11,East London,-33.0153,27.9116,69.80,88,0,9.22
118,Myitkyina,25.3833,97.4000,66.18,36,1,2.53
264,Hervey Bay,-25.2986,152.8535,69.01,87,0,1.01
324,Saldanha,-33.0117,17.9442,68.00,73,0,3.44
401,Faya,18.3851,42.4509,66.20,63,0,1.30
511,Valdivia,-39.8142,-73.2459,69.80,43,0,4.61
522,Emerald,-23.5333,148.1667,66.20,77,0,1.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates

hotel_df['Hotel Name'] = "" 
hotel_df['Country'] = ""
hotel_df['Coordinates'] = list(zip(hotel_df['Latitude'], hotel_df['Longitude']))

hotel_df = hotel_df[['Latitude','Longitude','City', 'Country', 'Hotel Name', 'Coordinates']]
hotel_df

<ipython-input-7-aa952b31a182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""
<ipython-input-7-aa952b31a182>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Country'] = ""
<ipython-input-7-aa952b31a182>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,Latitude,Longitude,City,Country,Hotel Name,Coordinates
11,-33.0153,27.9116,East London,,,"(-33.0153, 27.9116)"
118,25.3833,97.4000,Myitkyina,,,"(25.3833, 97.4)"
264,-25.2986,152.8535,Hervey Bay,,,"(-25.2986, 152.8535)"
324,-33.0117,17.9442,Saldanha,,,"(-33.0117, 17.9442)"
401,18.3851,42.4509,Faya,,,"(18.3851, 42.4509)"
511,-39.8142,-73.2459,Valdivia,,,"(-39.8142, -73.2459)"
522,-23.5333,148.1667,Emerald,,,"(-23.5333, 148.1667)"


In [8]:
# Set parameters to search for a hotel
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_radius = 5000
target_type = "lodging"
params = {
    "radius": target_radius,
    "type": target_type,
    "key": gkey
    }

# Iterate through 
for index, location in hotel_df.iterrows():

    # get lat, lng from df
    params["location"] = f"{location['Coordinates'][0]},{location['Coordinates'][1]}"

    # make request and print url
    print(f"Retrieving Results for {index}: {location}.")
    
    # convert to json
   
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    
    # Grab the first hotel from the results and store the name
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        #hotel_df.loc[index, 'Country'] = results[0]['country']
   
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    


Retrieving Results for 11: Latitude                  -33.0153
Longitude                  27.9116
City                   East London
Country                           
Hotel Name                        
Coordinates    (-33.0153, 27.9116)
Name: 11, dtype: object.
Closest hotel is Tu Casa.
------------
Retrieving Results for 118: Latitude               25.3833
Longitude                 97.4
City                 Myitkyina
Country                       
Hotel Name                    
Coordinates    (25.3833, 97.4)
Name: 118, dtype: object.
Closest hotel is Shwe Phyu Hotel.
------------
Retrieving Results for 264: Latitude                   -25.2986
Longitude                   152.853
City                     Hervey Bay
Country                            
Hotel Name                         
Coordinates    (-25.2986, 152.8535)
Name: 264, dtype: object.
Closest hotel is Shelly Bay Resort.
------------
Retrieving Results for 324: Latitude                  -33.0117
Longitude                  17.

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Latitude', 'Longitude']]

In [13]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add marker layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))